# Simple circuits with PennyLane

<div class="alert alert-warning">
    Got feedback? Share your thoughts by filling out our <a href="https://forms.gle/z6ochGoC6grMh65E9">survey</a>.
</div>

Hybrid algorithms consist of a mixture of classical and quantum computations.

<img src="fig/hybrid_graph.png" width=700>

In PennyLane, units of quantum computation are represented by quantum nodes, or QNodes. A QNode consists of two parts:

 - a quantum function
 - a device

<img src="fig/qnode.svg" width=700>

Suppose we want to run the following quantum circuit:

<img src="fig/circuit_1.svg" width=500>

## Devices

A device specifies where a quantum computation runs; this could be a simulator, or an actual hardware device. Here, we will use a simulator. 

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
dev = qml.device('default.qubit', wires=3)

## Quantum functions

A quantum function is a normal Python function that performs one or more quantum operations, and returns a measurement.

(A complete list of operations can be found in the [PennyLane documentation](https://pennylane.readthedocs.io/en/stable/introduction/operations.html))

<img src="fig/circuit_1.svg" width=300>

In [ ]:
def circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    
    return qml.expval(qml.PauliZ(wires=0))

## QNodes

We bind together the device and the quantum function to create a quantum node:

In [ ]:
our_qnode = qml.QNode(circuit, dev)

Now we can execute the QNode by calling it like a function; the quantum function will be executed on the device, and we will receive the outcome of the measurement back.

In [ ]:
theta = np.array(0.1)
phi = np.array(0.2)
omega = np.array(0.3)

In [ ]:
our_qnode(theta, phi, omega)

We can also draw our QNode using ``qml.draw``:

In [ ]:
print(qml.draw(our_qnode)(theta, phi, omega))

## The `qnode` decorator

There is also a "shortcut" for constructing QNodes using decorators.

<img src="fig/circuit_2.svg" width=500>

In [ ]:
@qml.qnode(dev)
def circuit_2(theta, phi):
    qml.CNOT(wires=[0, 1])
    qml.RX(theta, wires=2)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[2, 0])
    qml.RY(phi, wires=1)
    
    return qml.expval(qml.PauliY(wires=0)), qml.expval(qml.PauliZ(wires=1))

In [ ]:
circuit_2(theta, phi)

In [ ]:
print(qml.draw(circuit_2)(theta, phi))

## Plugins and devices

PennyLane offers [plugins](https://pennylane.ai/plugins.html) to a number of different hardware providers. You can swap out simulators for other devices, including real hardware, very easily!

In [ ]:
# If you have a token for IBM Q Experience, you can use their devices like so,
# where backend gets replaced with the backend of your choice.
# dev = qml.device('qiskit.ibmq', wires=3, backend='ibmq_qasm_simulator')

dev = qml.device('qiskit.aer', wires=3, shots=1000)

In [ ]:
circuit_with_shots = qml.QNode(circuit, dev)

In [ ]:
circuit_with_shots(theta, phi, omega)

Expectation values aren't the only thing you can measure - you can also compute output probabilities for each of the possible outcomes:

In [ ]:
def circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    
    return qml.probs(wires=0)

circuit_with_shots = qml.QNode(circuit, dev)
circuit_with_shots(theta, phi, omega)